# Number of Transactions per Person (not graphed)
_Not controlling for number of years in position or size of transaction._ For each official, we want their total number of transactions.  {'Sam': 5, 'Alex': 2424, ...}

In [1]:
from utils import increment_dictionary, sort_dictionary_by_values, path_csv, get_data
from csv_utils import make_csv
import pandas as pd 

def num_of_trans_per_person():
    title, rows = get_data()
    d={}
    
    for _, transaction in rows:
        d = increment_dictionary(d, transaction[title])

    filename = "num_of_trans_per_person"
    key_header = title
    value_header = "number_of_transactions"

    d = sort_dictionary_by_values(d)
    
    wd = make_csv(path_csv, filename, d, [key_header, value_header])
    df = pd.read_csv(wd)
    print(df.head(5))

    return d 

trans_per_person_res = num_of_trans_per_person()


               senator  number_of_transactions
0  David A Perdue , Jr                    2597
1      Thomas R Carper                     970
2   Sheldon Whitehouse                     693
3          Pat Roberts                     477
4      Susan M Collins                     449


# Number of Transactions per Person Controlled (not graphed)
_Divide number of transactions by number of years in official position.  Not controlling for size of transaction._

In [2]:
from wiki import wiki_search
import math 

def num_of_trans_per_person_controlled(rows):
    title, _ = get_data()
    d={}

    for person, num_of_trans in rows.items():
        official = wiki_search(person)       
        years_in_office = official.get_num_of_years()
        d[person] = math.ceil(num_of_trans/years_in_office) 

    filename = "num_of_trans_per_person_controlled"
    value_header = "avg_number_of_transactions"

    d = sort_dictionary_by_values(d)
        
    wd = make_csv(path_csv, filename, d, [title, value_header])
    print(pd.read_csv(wd).head(5))
    
    return d 

num_of_trans_per_person_controlled_res = num_of_trans_per_person_controlled(trans_per_person_res)
    

               senator  avg_number_of_transactions
0  David A Perdue , Jr                         433
1       Kelly Loeffler                         279
2  Thomas H Tuberville                         219
3           Rick Scott                          76
4     Shelley M Capito                          63


# Number of Transactions per Person Controlled w/Affiliation (not graphed)

In [3]:
from wiki import wiki_search
import math 

def num_of_trans_per_person_controlled_w_aff(rows):
    title, _ = get_data()
    d={}
    
    for person, val in rows.items():
        official = wiki_search(person)       
        d[person] = val, official.get_party()

    filename = "num_of_trans_per_person_controlled_w_aff"
    value_header = "avg_number_of_transactions"

    d = sort_dictionary_by_values(d)
        
    wd = make_csv(path_csv, filename, d, [title, value_header, "party"])
    print(pd.read_csv(wd).head(5))
    
    return d, pd.read_csv(wd).head(10)

num_of_trans_per_person_controlled_w_aff_res, top_10 = num_of_trans_per_person_controlled_w_aff(num_of_trans_per_person_controlled_res)
    

               senator  avg_number_of_transactions  \
0  David A Perdue , Jr                         433   
1       Kelly Loeffler                         279   
2  Thomas H Tuberville                         219   
3           Rick Scott                          76   
4     Shelley M Capito                          63   

                              party  
0  Republican Party (United States)  
1  Republican Party (United States)  
2  Republican Party (United States)  
3  Republican Party (United States)  
4  Republican Party (United States)  


# Number of Transactions per Person by Date (not graphed)

In [4]:
from utils import increment_dictionary_in_dictionary, path_csv
from csv_utils import make_csv_breakdown

def num_of_trans_per_person_per_date():    
    title, rows = get_data()
    d={}

    for _, transaction in rows:
        date = transaction['transaction_date']
        d = increment_dictionary_in_dictionary(d, transaction[title], date)

    filename = "num_of_trans_per_person_per_date"
    
    print(d)
    
    wd = make_csv_breakdown(path_csv, filename, d, title)
    print(pd.read_csv(wd).head(5))

num_of_trans_per_person_per_date()


{'Thomas H Tuberville': {'09/30/2021': 16, '09/29/2021': 9, '09/20/2021': 2, '09/13/2021': 4, '09/03/2021': 1, '09/08/2021': 1, '09/02/2021': 1, '08/30/2021': 3, '08/27/2021': 1, '08/18/2021': 2, '08/12/2021': 3, '08/10/2021': 2, '08/02/2021': 2, '08/20/2021': 1, '08/19/2021': 1, '08/11/2021': 1, '08/05/2021': 1, '07/20/2021': 3, '07/19/2021': 3, '07/09/2021': 2, '07/07/2021': 1, '07/30/2021': 1, '07/29/2021': 1, '07/28/2021': 1, '07/21/2021': 5, '07/14/2021': 2, '07/06/2021': 1, '07/01/2021': 1, '07/16/2021': 9, '03/29/2021': 44, '04/05/2021': 3, '01/25/2021': 1, '01/21/2021': 2, '01/15/2021': 1, '01/13/2021': 2, '01/06/2021': 5, '01/05/2021': 1, '01/04/2021': 7, '02/19/2021': 1, '02/16/2021': 6, '02/11/2021': 1, '02/10/2021': 8, '02/09/2021': 1, '02/08/2021': 1, '02/04/2021': 2, '02/03/2021': 2, '02/02/2021': 1, '03/16/2021': 4, '03/12/2021': 3, '03/11/2021': 1, '04/27/2021': 3, '04/26/2021': 1, '04/21/2021': 2, '04/20/2021': 2, '04/19/2021': 5, '04/16/2021': 2, '04/14/2021': 2, '04/

# Number of Transactions per Date (graphed)

In [5]:
from graph_utils import graph_csv 
from csv_utils import make_csv
from utils import path_html, increment_dictionary, path_csv, reversedict
import plotly.express as px

def num_of_trans_per_person_per_date():
    _, rows = get_data()
    d={}

    for _, transaction in rows:
        date = transaction['transaction_date']
        d = increment_dictionary(d, date)

    filename = "num_of_trans_per_person_per_date"
    key_header = "date"
    value_header = "number_of_transactions"
    
    d = reversedict(d)
    
    make_csv(path_csv, filename, d, [key_header, value_header])
    fig = graph_csv(path_csv, path_html, filename, key_header, value_header)
    fig.show()
    return d 

trans_per_person_per_date_res = num_of_trans_per_person_per_date()

# Frequency of Transactions per Date Controlled (graphed)
_Number of transactions per date controlled by official. E.g. if Ted Baker made 40 transactions on 1/1/02 and Sam Wall made 2 transactions on 1/1/02, we conclude that there were two transactions on 1/1/02._


In [6]:
from utils import increment_dictionary_in_dictionary, path_csv, path_html, reversedict, get_data
from csv_utils import make_csv
from graph_utils import graph_csv

def num_of_trans_per_date_controlled():
    title, rows = get_data()
    d = {}

    for _, transaction in rows:
        date = transaction['transaction_date']
        d = increment_dictionary_in_dictionary(d, date, transaction[title])
    
    d2 = {}
    for date in d:
        d2[date] =  len(d[date])
        
    d2 = reversedict(d2)
        
    filename = "num_of_trans_per_date_controlled"
    key_header = "date"
    value_header = "number_of_transactions_unique"

    make_csv(path_csv, filename, d2, [key_header, value_header])
    fig = graph_csv(path_csv, path_html, filename, key_header, value_header, scatter=True)
    fig.show()
    return d 
    
num_of_trans_per_date_controlled_res = num_of_trans_per_date_controlled()


# Number of Transactions Within 2 Weeks Prior to Quarterly Tax Date (not graphed)

In [7]:
from utils import within_tax_date, increment_dictionary
from wiki import wiki_search

def num_of_trans_within_tax_date(rows):
        total = within = 0 
        
        for k,v in rows.items():  
                total += 1 
                if within_tax_date(k):
                        within += v 

        print("Percent of transactions posted within two weeks of quarterly tax deadline: {percent}%".format(percent=str((within/total)*100)[:5]))
        return (within/total)*100

frac = num_of_trans_within_tax_date(trans_per_person_per_date_res)


Percent of transactions posted within two weeks of quarterly tax deadline: 81.11%


# Number of Transactions Within 2 Weeks Prior to Quarterly Tax Date Semi-Controlled (not graphed)
_Given dict='09/03/2021': {'Thomas H Tuberville': 1, 'Cynthia M Lummis': 1, 'A. Mitchell Mcconnell, Jr.': 1}...I only incremement the number of within (tax date) once per date per official. So, if an official does 100 transactions on a date within two weeks of a quarterly deadline, then I only count it as one transaction._

_A Note: total === number of transactions per person per date (so not really all transactions) because someone could have potentially made 60 transactions on one date which we don't include in neither total or within, if applicable._

In [8]:
def num_of_trans_within_tax_date_controlled(rows):
        total = within = 0 
        people = set()

        for date, inner_dict in rows.items():
                if within_tax_date(date):
                    for person in inner_dict:
                                people.add(person)
                                within += 1 
                                total += 1 
                else:
                        for person in inner_dict:
                                total += 1 
     
     
        print("Percent of transactions posted within two weeks of quarterly tax deadline: {percent}%".format(percent=str((within/total)*100)[:5]))
        return people

num_of_trans_within_tax_date_controlled_res = num_of_trans_within_tax_date_controlled(num_of_trans_per_date_controlled_res)


Percent of transactions posted within two weeks of quarterly tax deadline: 15.18%


In [9]:
def people_and_within_tax_date(people):        
        wd = make_csv(path_csv, "people_and_within_tax_date_list", people, ["Officials"])
        df = pd.read_csv(wd)
        print("People who posted transactions within two weeks of quarterly tax deadline:\n {}\n".format(df.head(5)))

        print("Number of people who posted transactions within two weeks of quarterly tax deadline: {}\n".format(len(people)))
        
        party = {}
        for p in people:
                party = increment_dictionary(party, wiki_search(p).get_party())
        
        wd = make_csv(path_csv, "people_and_within_tax_date_list_w_aff", party, ["party", "number_of_filing_within_tax_date"])
        df = pd.read_csv(wd)
        print("Party breakdown of people who posted transactions within two weeks of quarterly tax deadline:\n {}\n".format(df.head(5)))

people_and_within_tax_date(num_of_trans_within_tax_date_controlled_res)

People who posted transactions within two weeks of quarterly tax deadline:
            Officials
0     Kelly Loeffler
1     Ronald L Wyden
2      Mark R Warner
3        Roy D Blunt
4  Angus S King, Jr.

Number of people who posted transactions within two weeks of quarterly tax deadline: 47

Party breakdown of people who posted transactions within two weeks of quarterly tax deadline:
                               party  number_of_filing_within_tax_date
0  Republican Party (United States)                                24
1  Democratic Party (United States)                                22
2            Independent politician                                 1



In [10]:
def people_and_within_tax_date_how_often(people):
        title, rows = get_data()
        d = {}
        d_controlled_by_dates = {}
        for _, transaction in rows:
                if transaction[title] in people and within_tax_date(transaction['transaction_date']):
                        d = increment_dictionary(d, transaction[title])
                        d_controlled_by_dates = increment_dictionary_in_dictionary(d_controlled_by_dates, transaction['transaction_date'], transaction[title])

        d_controlled_by_dates_res  = {}
        for date in d_controlled_by_dates:
                for person in d_controlled_by_dates[date]:
                        d_controlled_by_dates_res = increment_dictionary(d_controlled_by_dates_res, person)

        d = sort_dictionary_by_values(d)
        d_controlled_by_dates_res = sort_dictionary_by_values(d_controlled_by_dates_res)

        wd = make_csv(path_csv, "people_and_within_tax_date_how_often", d, [title, "number_of_filing_within_tax_date"])
        df = pd.read_csv(wd)
        print("People who posted transactions within two weeks of quarterly tax deadline and the number of transactions posted:\n {}\n".format(df.head(5)))

        wd = make_csv(path_csv, "people_and_within_tax_date_how_often_date_controlled", d_controlled_by_dates_res, [title, "number_of_filing_within_tax_date_date_controlled"])
        df = pd.read_csv(wd)
        print("People who posted transactions within two weeks of quarterly tax deadline and the number of transactions posted controlled by date:\n {}\n".format(df.head(5)))

          
people_and_within_tax_date_how_often(num_of_trans_within_tax_date_controlled_res)



People who posted transactions within two weeks of quarterly tax deadline and the number of transactions posted:
                senator  number_of_filing_within_tax_date
0  David A Perdue , Jr                               368
1      Thomas R Carper                               179
2       Kelly Loeffler                               155
3     Shelley M Capito                               139
4   Sheldon Whitehouse                               102

People who posted transactions within two weeks of quarterly tax deadline and the number of transactions posted controlled by date:
                senator  number_of_filing_within_tax_date_date_controlled
0  David A Perdue , Jr                                               113
1      Thomas R Carper                                                52
2          Pat Roberts                                                44
3   Sheldon Whitehouse                                                28
4      Susan M Collins                       

# Frequency of Differences between Transaction and Disclosure Date

In [11]:
from utils import difference_between_dates, increment_dictionary, sort_dictionary_by_keys, path_csv, path_html
from graph_utils import graph_csv
from csv_utils import make_csv

def frequency_of_differences(d={}):
    title, rows = get_data()

    for _, transaction in rows:
        transaction_date = transaction['transaction_date']
        disclosure_date = transaction['disclosure_date']

        # Negative, X days BEFORE
        # Positive, Y dayas AFTER
        diff = difference_between_dates(disclosure_date, transaction_date)        
        d = increment_dictionary(d, int(diff))
    
    d = sort_dictionary_by_keys(d)
        
    filename = "frequency_of_differences"
    key_header = "difference_in_days"
    value_header = "#_of_transactions_with_that_diff"
    make_csv(path_csv, filename, d, [key_header, value_header])
    fig = graph_csv(path_csv, path_html, filename, key_header, value_header)
    fig.show()

frequency_of_differences()

# Frequency of Actions

In [12]:
from utils import get_data

def frequency_of_act(d={}):
    _, rows = get_data()

    for _, transaction in rows:
        d = increment_dictionary(d, transaction['type'])
    
    d = sort_dictionary_by_values(d)
    
    filename = "frequency_of_act"
    key_header = "type"
    value_header = "number_of_transactions"
    wd = make_csv(path_csv, filename, d, [key_header, value_header])
    df = pd.read_csv(wd)
    print(df.head(5))

     
frequency_of_act()

             type  number_of_transactions
0        Purchase                    4350
1     Sale (Full)                    2212
2  Sale (Partial)                    1756
3             NaN                     465
4        Exchange                      96


# Frequency of Asset Type

In [13]:
def frequency_of_asset_type(d={}):
    _, rows = get_data()

    for _, transaction in rows:
        d = increment_dictionary(d, transaction['asset_type'])
      
    d = sort_dictionary_by_values(d)
  
    filename = "frequency_of_asset_type"
    key_header = "asset_type"
    value_header = "number_of_transactions"
    wd = make_csv(path_csv, filename, d, [key_header, value_header])
    df = pd.read_csv(wd)
    print(df.head(5))


frequency_of_asset_type()

             asset_type  number_of_transactions
0                 Stock                    6486
1                   NaN                     666
2  PDF Disclosed Filing                     465
3    Municipal Security                     380
4      Other Securities                     359


# Frequency of Transactions by Political Affiliation

In [14]:
from wiki import wiki_search
from utils import increment_dictionary_in_dictionary, add_sort_key_for_amount, get_data, sort_dictionary_by_sort_key, path_csv 
from csv_utils import make_csv_breakdown
import pandas as pd 

def frequency_of_amount_by_aff(d={}):
    _, rows = get_data()

    affiliations = {}
    for _, transaction in rows:
        person = transaction['senator']
        
        if person not in affiliations: 
            rep = wiki_search(person) 
            affiliations.update({person : rep.party})
            
        d = increment_dictionary_in_dictionary(d, transaction['amount'], affiliations[person])

    
    d = add_sort_key_for_amount(d)
    d = sort_dictionary_by_sort_key(d)
    
        
    filename = "frequency_of_amount_by_aff"
    key_header = "amount"
    
    wd = make_csv_breakdown(path_csv, filename, d, key_header)
    print(pd.read_csv(wd).head(2))
   
    
frequency_of_amount_by_aff()

             amount  Democratic Party (United States)  Independent politician  \
0           Unknown                             189.0                     8.0   
1  $1,001 - $15,000                            1981.0                    51.0   

   Republican Party (United States)  sort_key  
0                               268        -1  
1                              3714     15000  


# Frequency of Tickers 
_Only tracking valid ones._

In [20]:
from utils import isvalid 
# need to group the popular ones !!! @TODO 
def frequency_of_ticker(d={}):
    _, rows = get_data()

    # Only tracking the valid ones. 
    for _, transaction in rows:
        if isvalid(transaction['ticker']):
            d = increment_dictionary(d, transaction['ticker'])

    filename = "ticker_total"
    key_header = "ticker"
    value_header = "number_of_transactions"
    make_csv(path_csv, filename, d, [key_header, value_header])
    fig = graph_csv(path_csv, path_html, filename, key_header, value_header) 
    fig.show()

frequency_of_ticker()

# Frequency of Ticker per Person 

In [21]:
def frequency_of_ticker_breakdown_person(d={}):
    _, rows = get_data()

    for _, transaction in rows:
        d = increment_dictionary_in_dictionary(d, transaction['senator'], transaction['ticker'])

    filename = "ticker_per_person_breakdown"
    key_header = "senator"

    wd = make_csv_breakdown(path_csv, filename, d, key_header)
    print(pd.read_csv(wd).head(2))

frequency_of_ticker_breakdown_person()

               senator  0QZI.IL  3V64.TI   A  AA  AAGIY  AAL  AAN  AAON  AAPL  \
0  Thomas H Tuberville      NaN      NaN NaN NaN    NaN  NaN  NaN   NaN   NaN   
1     Cynthia M Lummis      NaN      NaN NaN NaN    NaN  NaN  NaN   NaN   NaN   

   ...  ZAYO  ZBH  ZIOP  ZM  ZMH  ZNGA  ZNGA.SW  ZTS  ^MWE  ^RGP  
0  ...   NaN  NaN   NaN NaN  NaN   NaN      NaN  NaN   NaN   NaN  
1  ...   NaN  NaN   NaN NaN  NaN   NaN      NaN  NaN   NaN   NaN  

[2 rows x 1093 columns]


# Frequency of Ticker per Year

In [22]:
from utils import get_year, increment_dictionary_in_dictionary, path_csv
from csv_utils import make_csv_breakdown

def frequency_of_ticker_breakdown_ticker(d={}):
    _, rows = get_data()

    for _, transaction in rows:
        d = increment_dictionary_in_dictionary(d, transaction['ticker'], get_year(transaction['transaction_date']))

    filename = "trans_per_year_breakdown"
    key_header = "ticker"
    
    wd = make_csv_breakdown(path_csv, filename, d, key_header)
    print(pd.read_csv(wd).head(2))

frequency_of_ticker_breakdown_ticker()

  ticker  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021
0     PG   NaN   NaN   1.0   6.0   8.0   5.0  10.0   6.0   7.0   1.0
1   NFLX   NaN   NaN   9.0   8.0   6.0   7.0  14.0   6.0  27.0   1.0


# Frequency of Ticker per Date

In [18]:
from utils import get_year, increment_dictionary_in_dictionary, path_csv
from csv_utils import make_csv_breakdown

def frequency_of_ticker_by_date(d={}):
    _, rows = get_data()
    
    for _, transaction in rows:
        d = increment_dictionary_in_dictionary(d, transaction['ticker'], transaction['transaction_date'])

    filename = "frequency_of_ticker_by_date"
    key_header = "ticker"
    
    wd = make_csv_breakdown(path_csv, filename, d, key_header)
    print(pd.read_csv(wd).head(2))

frequency_of_ticker_by_date()

  ticker  01/01/2014  01/02/2013  01/02/2014  01/02/2015  01/02/2018  \
0     PG         NaN         NaN         NaN         NaN         NaN   
1   NFLX         NaN         NaN         NaN         NaN         NaN   

   01/02/2020  01/03/2014  01/03/2017  01/03/2018  ...  12/30/2014  \
0         NaN         NaN         NaN         NaN  ...         NaN   
1         NaN         NaN         NaN         NaN  ...         NaN   

   12/30/2015  12/30/2016  12/30/2019  12/30/2020  12/31/2014  12/31/2015  \
0         NaN         NaN         NaN         NaN         NaN         NaN   
1         NaN         NaN         NaN         NaN         NaN         NaN   

   12/31/2018  12/31/2019  12/31/2020  
0         NaN         NaN         NaN  
1         NaN         NaN         NaN  

[2 rows x 1833 columns]


# Number of Transactions per Industry
_Not controlled in any way._

In [4]:
# breakdown by year 
from utils import increment_dictionary, sort_dictionary_by_values, path_csv, get_data, isvalid
from csv_utils import make_csv
from search import get_industry
import pandas as pd 

# have to save this because way too slow 
def number_of_transactions_per_indusry():
    d = {}
    _, rows = get_data()
    cache = {}
    
    for _, transaction in rows:
        ticker = transaction['ticker']
        if isvalid(ticker):
            
            if ticker not in cache: 
                cache[ticker] = get_industry(ticker)        
            
            print(len(cache))
            d = increment_dictionary(d, cache[ticker])

    filename = "number_of_transactions_per_indusry"
    key_header = "industry"
    value_header = "number_of_transactions"

    d = sort_dictionary_by_values(d)
    
    wd = make_csv(path_csv, filename, d, [key_header, value_header])

    df = pd.read_csv(wd)
    print(df.head(5))

    return d 

number_of_transactions_per_indusry()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
27
27
27
28
29
30
31
32
33
33
34
34
34
35
36
37
37
37
37
37
SSBK
38
39
40
40
41
42
43
44
45
46
47
48
48
48
48
48
48
48
48
48
48
48
QQQ
49
49
CRWD
50
51
52
53
XLY
54
55
56
DTD
57
58
58
59
59
60
61
62
63
64
IVV
65
ABYIX
66
CFA
67
GLD
68
FDN
69
LUBFX
70
DGRS
71
GBIL
72
73
NXTG
74
FIMKX
75
GEMIX
76
FCPIX
77
78
79
80
81
82
82
82
82
82
SDOG
83
83
84
84
84
84
84
84
85
85
85
85
85
86
ONEQ
87
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
88
89
89
89
89
89
90
91
92
92
93
94
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
108
108
109
110
110
111
112
113
114
115
116
117
118
118
119
120
120
120
121
122
122
122
122
123
123
123
123
123
123
123
123
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
125
125
125
125
125
125
125
125
125
125
125
125
125
FDSWX
126
126
126
126
126
126
126
126
127
128
128
128
128
128
128
128
129
129
130
131
ETHE
132
GBTC
133
134
134
135
135
136
137
138
139
139
140
IWM
141
142
143

# EXTRA CODE

In [19]:
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Finds missing dates in a dictionary (assuming the dates are keys) and adds it into the dictionary with value of 0.
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# def get_missing_dates(dict):
#     l = list(dict.keys())
#     today = dt.date.today()

#     while l:
#         start_date = l.pop()
#         #     # if start_date.weekday() > 4:
#         #     print("Weekend")

#         start_date_obj = dt.datetime.strptime(start_date, "%m/%d/%Y") 
        
#         try: 
#             next_date_obj = (start_date_obj + dt.timedelta(days=1))
#             next_date_str = next_date_obj.strftime("%m/%d/%Y")
#         except OverflowError:
#             continue 
        
#         if dt.date(next_date_obj.year, next_date_obj.month, next_date_obj.day) > today: 
#             continue 
        
#         if next_date_str not in dict:
#             dict = add_key_dictionary(dict, next_date_str)
#             l.append(next_date_str)
            
# # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#         from utils import increment_dictionary, sort_dictionary_by_values, path_csv
# from csv_utils import make_csv_tuples
# import plotly.express as px
# from wiki import wiki_search

# def num_of_trans_per_person_w_aff(trans_per_person_res, d={}, aff={}):
#     res = {}
#     for k, v in trans_per_person_res.items():
#         if k not in aff:
#             aff[k] = wiki_search(k)
#         res[k] = (v, aff[k].get_party(), aff[k].get_num_of_years())
        
#     filename = "num_of_trans_per_person_with_aff"
#     key_header = title
#     value_header = "number_of_transactions"
#     value_header2 = "affiliation"
#     value_header3 = "number of years in congress"

    
#     wd = make_csv_tuples(path_csv, filename, res, key_header, value_header, value_header2, value_header3)
#     df = pd.read_csv(wd)
#                 #  hover_name=value_header , log_x=True, 
#     fig = px.scatter(df, x=value_header3, y=value_header, 
# 	         color=value_header2, size=value_header, size_max=60)
#     fig.show()

#     return d 

# trans_per_person_res = num_of_trans_per_person_w_aff(trans_per_person_res)

IndentationError: unexpected indent (2040678870.py, line 29)